**The information used in this notebook is adopted from *Speech and Language Processing*. Jurafsky, D. & Martin, J. H. Draft of August 7, 2017. Chapter 2.**

# Dynamic Programming

Dynamic programming is the name for a class of algorithms, first introduced by Bellman (1957), that apply a table-driven method to solve problems by combining solutions to sub-problems. Example: MED, Viterbie, CKY etc. 

The intuition of a dynamic programming problem is that a large problem can be solved by properly combining the solutions to various sub-problems.

(Jurafsky and Martin 2017:Chapter 2 p19)

# Minimum Edit Distance

### MED introduction

The minimum edit distance between two strings is defined as the minimum number of editing operations (operations like insertion, deletion, substitution) needed to transform one string into another.

We can assign a particular cost or weight to each of these operations.

<img src="img/img1.png" alt="Drawing" style="width: 500px;"/>

### MED formal definition

Given two strings, the source string *X* of length *n*, and target string *Y* of length *m*, we'll define *D(i,j)* as the edit distance between *X[1...i]* and *Y[1...j]*, i.e., the first *i* characters of *X* and the first *j* characters of *Y*. The edit distance between *X* and *Y* is thus *D(n,m)*.

Dynamic programming will be used to computer *D(n,m)* bottom up, combining solutions to subproblems.

**In the base case**, with a source substring of length *i* but an empty target string, going from *i* character to 0 requires *i* deletion. With a target substring of length *j* but an empty source going from 0 characters to *j* characters requires *j* insertion. **Having computed *D(i,j)* for small *i*, *j* ** we then compute larger *D(i,j)* based on previously computed smaller values. **The value of *D(i,j)* **is computed by taking the minimum of the three possible paths through the matrix which arrive there:


<img src="img/img2.png" alt="Drawing" style="width: 300px;"/>

If we assume insertion and deletion each has a cost of 1 and substitution has a cost of 2 (i.e. the setting for Levenshtein distance), the computation of *D(i,j)* becomes:

<img src="img/img3.png" alt="Drawing" style="width: 300px;"/>

Applying the algorithm with operation cost above to calculate the distance between *intention* and *execution*, we get the following matrix:

<img src="img/img4.png" alt="Drawing" style="width: 500px;"/>

Pseudo-code for the algorithm:

<img src="img/img5.png" alt="Drawing" style="width: 400px;"/>

### MED for Alignment

The minimum edit distance algorithm can be extended for alignment by storing backpointers in each cell.

**The computaton proceeds in two steps:**

In the first step, we augment the MED algorithm to store backpointers in each cell. The backpointer from a cell points to the previous cell/cells that we came from in entering the current cell. Some cells have multiple backpointers because the minimum extension could have come from multiple previous cells.

In the second step, we perform a **backtrace**. In a backtrace, we start from the last cell (at the final row and column), and follow the pointer back through the dynamic programming matrix. 

Each complete path between the final cell and the inital cell is a minimum distance alignment.

<img src="img/img6.png" alt="Drawing" style="width: 500px;"/>

# Exercise:

### Part 1: Basic Implementation of MED with fixed costs

Define a function called *MED* to implement a menimum edit distance algorithm with the operation cost for insertion and deletion as 1 and for substituion as 2. The function takes as input a source string and a target string, and returns the MED between the two strings.

For example, when you call *MED* with *intention* and *execution* with *MED('intension', 'execution')*, it will return 8.

### Part 2: Implementation of MED with flexible costs

Improve your function *MED* in Part 1 to allow for assigning different costs for different operation. After improvement, the *MED* function should take as input the source string, the target string, insertion cost, deletion cost, and substitution cost, and return the MED between the two strings.

For example, calling the function by

*MED('intention', 'execution', 1, 1, 2)*

should return 

*8*

### Part 3: MED for Alignment

Augment the *MED* function part 2 to output an alignment. After the augmentation, your *MED* function still takes as input the source string, the target string, insertion cost, deletion cost, and substitution cost, but it should return not only the MED between the two stirngs but also the alignment of the two strings.

For example, calling the function by

*MED('intention', 'execusion', 1, 1, 2)*

should return 

*8 inte_ntion _execution*

### Example Codes to the Exercise:

NumPy

Numpy is the core library for scientific computing in Python. It provides a high-performance multidimensional array object, and tools for working with these arrays. 

http://cs231n.github.io/python-numpy-tutorial/#numpy


#### Part 1

In [30]:
import numpy as np

def MED(source, target):
    m = len(target) + 1
    n = len(source) + 1
    mtx = np.zeros([n, m]) #define a n*m matrix
    #print (mtx)
    for i in range(1, n):
        mtx[i,0] = mtx[i-1, 0] + 1 #deletion
    for j in range(1, m):
        mtx[0,j] = mtx[0, j-1] + 1 #insertion
    for i in range(1, n):
        for j in range(1, m):
            if target[j-1] == source[i-1]:
                sb = 0 #identical--self-substitution
            else:
                sb = 2 #substitution
            mtx[i, j] = min(mtx[i-1, j] + 1,
                            mtx[i, j-1] + 1,
                            mtx[i-1, j-1] + sb)
            #deletion
            #insertion
            #substitution
    #print (mtx)
    med = mtx[len(source), len(target)]
    return med

In [31]:
print (MED('intention', 'execution'))

8.0


#### Part 2

In [28]:
import numpy as np

def MED(source, target, insert = 1, delete = 1, substitute = 2):
    m = len(target) + 1
    n = len(source) + 1
    mtx = np.zeros([n, m]) #define a n*m matrix
    #print (mtx)
    for i in range(1, n):
        mtx[i,0] = mtx[i-1, 0] + delete #deletion
    for j in range(1, m):
        mtx[0,j] = mtx[0, j-1] + insert #insertion
    for i in range(1, n):
        for j in range(1, m):
            if target[j-1] == source[i-1]:
                sb = 0 #identical--self-substitution
            else:
                sb = substitute #substitution
            mtx[i, j] = min(mtx[i-1, j] + delete,
                            mtx[i, j-1] + insert,
                            mtx[i-1, j-1] + sb)
            #deletion
            #insertion
            #substitution
    #print (mtx)
    med = mtx[len(source), len(target)]
    return med

In [29]:
print (MED('intention', 'execution', 1, 1, 1))

5.0


#### Part 3

In [35]:
import numpy as np


def MED(source, target, insert = 1, delete = 1, substitute = 2):
    """MED with Backpointer"""
    
    m = len(target) + 1
    n = len(source) + 1
    mtx = np.zeros([n, m]) #define a n*m matrix
    #print (mtx)
    bpointer = np.zeros(mtx.shape, dtype = 'int, int') 
    #define a matrix of the same dimensions as mtx, 
    #but its elements are two-element tuples of integers
    #print (bpointer)
    for i in range(1, n):
        mtx[i,0] = mtx[i-1, 0] + delete #deletion
    for j in range(1, m):
        mtx[0,j] = mtx[0, j-1] + insert #insertion
    for i in range(1, n):
        for j in range(1, m):
            if target[j-1] == source[i-1]:
                sb = 0 #identical--self-substitution
            else:
                sb = substitute #substitution
            mtx[i, j], bpointer[i, j] = min((mtx[i-1, j] + delete, (i-1, j)), 
                                            (mtx[i, j-1] + insert, (i, j-1)),
                                            (mtx[i-1, j-1] + sb, (i-1, j-1)))
            #deletion
            #insertion
            #substitution
    #print (mtx)
    print (bpointer)
    med = mtx[len(source), len(target)]
    return med

In [36]:
print (MED('intention', 'execution'))

[[(0, 0) (0, 0) (0, 0) (0, 0) (0, 0) (0, 0) (0, 0) (0, 0) (0, 0) (0, 0)]
 [(0, 0) (0, 0) (0, 1) (0, 2) (0, 3) (0, 4) (0, 5) (0, 6) (1, 7) (1, 8)]
 [(0, 0) (1, 0) (1, 1) (1, 2) (1, 3) (1, 4) (1, 5) (1, 7) (1, 7) (1, 8)]
 [(0, 0) (2, 0) (2, 1) (2, 2) (2, 3) (2, 4) (2, 5) (2, 7) (2, 7) (2, 9)]
 [(0, 0) (3, 0) (4, 1) (3, 2) (4, 3) (4, 4) (3, 6) (3, 6) (3, 7) (3, 9)]
 [(0, 0) (4, 1) (4, 1) (4, 2) (4, 3) (4, 4) (4, 5) (4, 6) (4, 7) (4, 8)]
 [(0, 0) (5, 1) (5, 1) (5, 2) (5, 3) (5, 4) (5, 5) (6, 6) (6, 7) (5, 9)]
 [(0, 0) (6, 1) (6, 1) (6, 2) (6, 3) (6, 4) (6, 6) (6, 6) (7, 7) (7, 8)]
 [(0, 0) (7, 1) (7, 1) (7, 2) (7, 3) (7, 4) (7, 6) (7, 7) (7, 7) (8, 8)]
 [(0, 0) (8, 1) (8, 1) (8, 2) (8, 3) (8, 4) (8, 6) (8, 7) (8, 8) (8, 8)]]
8.0


In [ ]:
def alignment(src, tgt, source, target, i, j, m, n, bp):
    
    """Align the two strings with the path provided by the back pointer"""
    
    if m == 0 and n == 0: 
        #the x and y-axis are all (0, 0) along
        #reach this means reach the beginning
        return src[::-1], tgt[::-1]
    if i+1 == m and j+1 == n: 
        #substitute
        src += source[m-1]
        tgt += target[n-1]
    elif i == m and j+1 == n:
        #insert to tgt, 
        #relative src position is empty at the first place
        src += '_'
        tgt += target[n-1]
    elif i+1 == m and j == n:
        #from above, delete from src,
        #relative tgt position is empty at first place
        src += source[m-1]
        tgt += '_'
    print (src[::-1])
    print (tgt[::-1])
    print ()
    #m = i
    #n = j
    #newi = bp[i, j][0]
    #newj = bp[i, j][1]
    #i = newi
    #j = newj
    return alignment(src, tgt, source, target, *bp[i, j], i, j, bp)